<a href="https://colab.research.google.com/github/satanjj10/TKB_LineBot/blob/main/Demo_%E6%94%BE%E6%B4%8BLineBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 此處不動

!pip install line-bot-sdk flask flask-ngrok google-cloud-logging

from flask import Flask, request, abort, jsonify
import json
from flask_ngrok import run_with_ngrok

from linebot import (
    LineBotApi, WebhookHandler
)

from linebot.exceptions import (
    InvalidSignatureError
)

from linebot.models import *

from linebot.models.template import *

from linebot.models.template import *

from linebot.models import (
    MessageEvent, TextMessage, PostbackEvent
)
import requests
import urllib.request
import os

from linebot.models.events import (
    FollowEvent
)

import pandas as pd
import datetime
from csv import writer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# 設定Server啟用細節
# material為如果要找的到照片，可以建立一個material資料夾，把圖片放入
app = Flask(__name__,static_url_path = "/material" , static_folder = "./material/")
run_with_ngrok(app)

# 生成實體物件
line_bot_api = LineBotApi("LG+AB1pEGC4U6qLFXYsbYEADiFYYGjIzSSiPBbwJrhMsEriGRgT1I23t2AIWGkZpX7iQcuT2mFapVSetiKvLuUAx8LuDdTrl/acOcqe7y2U+UEfPfFMgbmQOJUJ5sZA43vJES5nSTfE1qqjTu/cXnQdB04t89/1O/w1cDnyilFU=")
handler = WebhookHandler("c7b8231cbb1aec69d8e647b389982f8d")

In [3]:
# 用瀏覽器訪問ngrok的url -> 導引此方法
@app.route("/callback", methods=['POST'])

def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)
    print(body)

    # 記錄用戶log
    # 把用戶傳來的消息，存在ai_event.log
    with open('event.csv', 'a') as f_object:
      writer_object = writer(f_object)
      writer_object.writerow(body)
      f_object.close()
    # f = open("event.log", "a")
    # f.write(body)
    # # f.write('\n')
    # f.close()

    df_message = pd.DataFrame(columns=['userId', 'timestamp', 'event_type','message_type','message_id', 'message_text'])

    for row in body:
      if json.loads(row).get('events'):
        events = json.loads(row).get('events')[0]
        print(json.loads(row).get('events')[0])
        
        if events.get('type') not in ['follow','unfollow','postback']:
          df_message = df_message.append(
              {
                'userId': events.get('source').get('userId'), 
                'timestamp': events.get('timestamp'), 
                'event_type': events.get('type'),
                'message_type': events.get('message').get('type'),
                'message_id': events.get('message').get('id'),
                'message_text': events.get('message').get('text')
              },
              ignore_index=True)
        elif events.get('type')  in ['postback']:
          df_message =df_message.append(
              {
                'userId': events.get('source').get('userId'), 
                'timestamp': events.get('timestamp'), 
                'event_type': events.get('type'),
                'message_text': events.get('postback').get('data')
              },
              ignore_index=True
          )
        else:
          df_message = df_message.append(
              {
                'userId': events.get('source').get('userId'), 
                'timestamp': events.get('timestamp'), 
                'event_type': events.get('type')
              },
              ignore_index=True)
          
    user_info_df= pd.DataFrame(columns=['userId', 'display_name', 'picture_url','status_message'])

    for user_id in df_message['userId'].unique():
      try:
        line_user_profile = line_bot_api.get_profile(user_id)
        
        user_info_df = user_info_df.append(
            {
              'display_name': line_user_profile.display_name, 
              'userId': line_user_profile.user_id, 
              'picture_url': line_user_profile.picture_url,
              'status_message': line_user_profile.status_message
            },
            ignore_index=True)
      except:
        pass
      
      merge_data = pd.merge(df_message, user_info_df)
      human_use_date=[]
      for stamp in merge_data['timestamp']:
        
        timestamp = datetime.datetime.fromtimestamp(stamp/1000+8*3600)
        human_use_date.append(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
        

      merge_data['human_date']=human_use_date
      merge_data.to_csv("log_all.csv", mode='a', index=False, header=False)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'

In [4]:
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 存取log檔
    user_id = event.source.user_id
    line_user_profile = line_bot_api.get_profile(user_id)
    
    # user_info的個人資訊
    user_info_df= pd.DataFrame(columns=['userId', 'display_name', 'picture_url','status_message'])
    user_info_df = user_info_df.append(
        {
          'display_name': line_user_profile.display_name, 
          'userId': line_user_profile.user_id, 
          'picture_url': line_user_profile.picture_url,
          'status_message': line_user_profile.status_message
        },
        ignore_index=True)
    
    # message的內容存取
    df_message = pd.DataFrame(columns=['userId', 'timestamp', 'event_type','message_type','message_id', 'message_text'])
    df_message = df_message.append(
          {
            'userId': event.source.user_id, 
            'timestamp': event.timestamp, 
            'event_type': event.type,
          },
          ignore_index=True)
      
    merge_data_followevent = pd.merge(df_message, user_info_df)
    
    human_use_date=[]
    for stamp in merge_data_followevent['timestamp']:
  
      timestamp = datetime.datetime.fromtimestamp(stamp/1000+8*3600)
      human_use_date.append(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
    
    merge_data_followevent['human_date']=human_use_date 

    merge_data_followevent.to_csv("log_followevent.csv", mode='a', index=False, header=False)   

    # 加入好友訊息
    line_bot_api.reply_message(event.reply_token,
        [
            TextSendMessage('感謝您加入放洋留遊學！\n\n我是XXX(吉祥物)，身為你的專屬客服，讓我來為你解惑吧!\n\n這裡會由我公告留遊學最新通知、活動、優惠，也可以和我一對一對話，還可以點選下方打字機圖示，了解更多喔~'),
            TextSendMessage('花個10秒鐘，完成以下選單，即可獲得免費資深專家約談喔!\n\n先讓我認識你吧~'),
            # LocationSendMessage(
            #                     title='服務據點',
            #                     address='台北市忠孝西路一段 102 號 3F',
            #                     latitude=25.04717,
            #                     longitude=121.51264),
            TemplateSendMessage(
                                alt_text='想去哪個國家留學?',
                                template=ImageCarouselTemplate(
                                                                columns=[
                                                                          ImageCarouselColumn(
                                                                          image_url='https://cdn.pixabay.com/photo/2021/11/02/18/58/airline-6763903_1280.png',
                                                                          action=MessageAction(
                                                                          label='我想去美國',
                                                                          text='我想去美國')
                                                              ),
                                                                          ImageCarouselColumn(
                                                                          image_url='https://cdn.pixabay.com/photo/2021/11/02/18/58/airline-6763903_1280.png',
                                                                          action=MessageAction(
                                                                          label='我想去英國',
                                                                          text='我想去英國')
                                                              ),
                                                                          ImageCarouselColumn(
                                                                          image_url='https://cdn.pixabay.com/photo/2021/11/02/18/58/airline-6763903_1280.png',
                                                                          action=MessageAction(
                                                                          label='我想去澳洲',
                                                                          text='我想去澳洲')
                                                              ),
                                                                          ImageCarouselColumn(
                                                                          image_url='https://cdn.pixabay.com/photo/2021/11/02/18/58/airline-6763903_1280.png',
                                                                          action=MessageAction(
                                                                          label='我想去加拿大',
                                                                          text='我想去加拿大')
                                                              ),]

                                ))
         ]
    )
     

In [5]:
@handler.add(UnfollowEvent)
def unfollowevent_get_user_profile(event):
    
    # 存取log檔
    user_id = event.source.user_id
    line_user_profile = line_bot_api.get_profile(user_id)
    
    # user_info的個人資訊
    user_info_df= pd.DataFrame(columns=['userId', 'display_name', 'picture_url','status_message'])
    user_info_df = user_info_df.append(
        {
          'display_name': line_user_profile.display_name, 
          'userId': line_user_profile.user_id, 
          'picture_url': line_user_profile.picture_url,
          'status_message': line_user_profile.status_message
        },
        ignore_index=True)
    
    # message的內容存取
    df_message = pd.DataFrame(columns=['userId', 'timestamp', 'event_type','message_type','message_id', 'message_text'])
    df_message = df_message.append(
          {
            'userId': event.source.user_id, 
            'timestamp': event.timestamp, 
            'event_type': event.type,
          },
          ignore_index=True)
      
    merge_data_unfollowevent = pd.merge(df_message, user_info_df)
    
    human_use_date=[]
    for stamp in merge_data_unfollowevent['timestamp']:
  
      timestamp = datetime.datetime.fromtimestamp(stamp/1000+8*3600)
      human_use_date.append(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
    
    merge_data_unfollowevent['human_date']=human_use_date 

    merge_data_unfollowevent.to_csv("log_unfollowevent.csv", mode='a', index=False, header=False) 

In [6]:
def detect_json_array_to_new_message_array(fileName):    
    
    #開啟檔案，轉成json
    with open(fileName,encoding='utf8') as f:
        jsonArray = json.load(f)
    
    # 解析json
    """
    先做一個空白的array
    判斷裡面的json的type，依據type欄位轉換成xxxSendMessage消息物件
    append 先前的空白array
    """
    returnArray = []
    for jsonObject in jsonArray:

        # 讀取其用來判斷的元件
        message_type = jsonObject.get('type')
        
        # 轉換
        if message_type == 'text':
            returnArray.append(TextSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'imagemap':
            returnArray.append(ImagemapSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'template':
            returnArray.append(TemplateSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'image':
            returnArray.append(ImageSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'sticker':
            returnArray.append(StickerSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'audio':
            returnArray.append(AudioSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'location':
            returnArray.append(LocationSendMessage.new_from_json_dict(jsonObject))
        elif message_type == 'flex':
            returnArray.append(FlexSendMessage.new_from_json_dict(jsonObject))  
        elif message_type == 'video':
            returnArray.append(VideoSendMessage.new_from_json_dict(jsonObject))    


    # 回傳
    return returnArray

In [7]:
@handler.add(MessageEvent,message=TextMessage)
def process_text_message(events):

    # 存取log檔
    user_id = events.source.user_id
    line_user_profile = line_bot_api.get_profile(user_id)
    
    # user_info的個人資訊
    user_info_df= pd.DataFrame(columns=['userId', 'display_name', 'picture_url','status_message'])
    user_info_df = user_info_df.append(
        {
          'display_name': line_user_profile.display_name, 
          'userId': line_user_profile.user_id, 
          'picture_url': line_user_profile.picture_url,
          'status_message': line_user_profile.status_message
        },
        ignore_index=True)
    

    # message的內容存取
    df_message = pd.DataFrame(columns=['userId', 'timestamp', 'event_type','message_type','message_id', 'message_text'])
    df_message = df_message.append(
          {
            'userId': events.source.user_id, 
            'timestamp': events.timestamp, 
            'event_type': events.type,
            'message_type': events.message.type,
            'message_id': events.message.id,
            'message_text': events.message.text
          },
          ignore_index=True)
    
    merge_data_messageevent = pd.merge(df_message, user_info_df)
    
    human_use_date=[]
    for stamp in merge_data_messageevent['timestamp']:
  
      timestamp = datetime.datetime.fromtimestamp(stamp/1000+8*3600)
      human_use_date.append(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
    
    merge_data_messageevent['human_date']=human_use_date

    merge_data_messageevent.to_csv("log_messageevent.csv", mode='a', index=False, header=False)

    # 設定取josn檔作為回覆
    # 讀取本地檔案，並轉譯成消息
    result_message_array =[]

    # 用戶輸入文字，就在本地端找相對應的json檔
    replyJsonPath = events.message.text+".json"
    result_message_array = detect_json_array_to_new_message_array(replyJsonPath)

    # 發送
    line_bot_api.reply_message(
        events.reply_token,
        result_message_array
    )

In [8]:
@handler.add(PostbackEvent)
def process_postback_message(events):

    # 存取log檔
    user_id = events.source.user_id
    line_user_profile = line_bot_api.get_profile(user_id)
    
    # user_info的個人資訊
    user_info_df= pd.DataFrame(columns=['userId', 'display_name', 'picture_url','status_message'])
    user_info_df = user_info_df.append(
        {
          'display_name': line_user_profile.display_name, 
          'userId': line_user_profile.user_id, 
          'picture_url': line_user_profile.picture_url,
          'status_message': line_user_profile.status_message
        },
        ignore_index=True)
    

    # message的內容存取
    df_message = pd.DataFrame(columns=['userId', 'timestamp', 'event_type','message_type','message_id', 'message_text'])
    df_message = df_message.append(
          {
            'userId': events.source.user_id, 
            'timestamp': events.timestamp, 
            'event_type': events.type,
            'message_text': events.postback.data
          },
          ignore_index=True)
    
    merge_data_postbackevent = pd.merge(df_message, user_info_df)
    
    human_use_date=[]
    for stamp in merge_data_postbackevent['timestamp']:
  
      timestamp = datetime.datetime.fromtimestamp(stamp/1000+8*3600)
      human_use_date.append(timestamp.strftime('%Y-%m-%d %H:%M:%S'))
    
    merge_data_postbackevent['human_date']=human_use_date

    merge_data_postbackevent.to_csv("log_postbackevent.csv", mode='a', index=False, header=False)

In [9]:
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://aa78-34-85-155-12.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


ERROR:__main__:Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-3-bb00406e85e1>", line 24, in callback
    if json.loads(row).get('events'):
  File "/usr/lib/python3.8/jso

{"destination":"U27545d4bec88860a1fb4b67195c22408","events":[]}


...

{"destination":"U27545d4bec88860a1fb4b67195c22408","events":[{"type":"postback","postback":{"data":"richmenu-changed-to-a","params":{"newRichMenuAliasId":"richmenu_a","status":"SUCCESS"}},"webhookEventId":"01GMAH3S0M15M2EEVNYTSVV63V","deliveryContext":{"isRedelivery":false},"timestamp":1671095771678,"source":{"type":"user","userId":"Uf57398b2870127a0567cc1fc518d93e0"},"replyToken":"4509b203f144404fb4654ce711837c9b","mode":"active"}]}


ERROR:__main__:Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-3-bb00406e85e1>", line 24, in callback
    if json.loads(row).get('events'):
  File "/usr/lib/python3.8/jso

{"destination":"U27545d4bec88860a1fb4b67195c22408","events":[{"type":"message","message":{"type":"text","id":"17298906307799","text":"我想去加拿大"},"webhookEventId":"01GMAH3SVBDG0211EKZ9GX5VAW","deliveryContext":{"isRedelivery":false},"timestamp":1671095772629,"source":{"type":"user","userId":"Uf57398b2870127a0567cc1fc518d93e0"},"replyToken":"3d08203f78474c0498d7c97563e68dca","mode":"active"}]}


ERROR:__main__:Exception on /callback [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "/usr/local/lib/python3.8/dist-packages/flask/_compat.py", line 39, in reraise
    raise value
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1936, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "<ipython-input-3-bb00406e85e1>", line 24, in callback
    if json.loads(row).get('events'):
  File "/usr/lib/python3.8/jso

{"destination":"U27545d4bec88860a1fb4b67195c22408","events":[{"type":"message","message":{"type":"text","id":"17298906404225","text":"我想去澳洲"},"webhookEventId":"01GMAH3V4EQZHFHJQP0ZYV521C","deliveryContext":{"isRedelivery":false},"timestamp":1671095773843,"source":{"type":"user","userId":"Uf57398b2870127a0567cc1fc518d93e0"},"replyToken":"3cf3ccded1104a4d8ae8edc3c03920ef","mode":"active"}]}


In [10]:
# @handler.add(MessageEvent, message=TextMessage)
# def handle_message(event):
#     message = text=event.message.text
#     if (event.message.text.find('我想去美國')== 0):       
#         line_bot_api.reply_message(event.reply_token, [
#                                                   TemplateSendMessage(                              
#                                                                       alt_text='想要留學還是短期遊學?',
#                                                                       template=ButtonsTemplate(
#                                                                                               title='想要留學還是短期遊學?',
#                                                                                               actions=[
#                                                                                                   MessageAction(
#                                                                                                       label='留學',
#                                                                                                       text='我想留學'), 
#                                                                                                   MessageAction(
#                                                                                                       label='短期遊學',
#                                                                                                       text='我想遊學'), ]))])


#     elif (event.message.text.find('我想留學')== 0):       
#         line_bot_api.reply_message(event.reply_token, [
#                                                   TemplateSendMessage(
#                                                                       alt_text='我的英文程度',
#                                                                       template=ButtonsTemplate(
#                                                                                                 title='我的英文程度',
#                                                                                                 text='請點選以下按鈕',
#                                                                                                 actions=[                                                                                            
#                                                                                                     MessageAction(
#                                                                                                         label='高',
#                                                                                                         text='我的英文頂呱呱'),
#                                                                                                     MessageAction(
#                                                                                                         label='中',
#                                                                                                         text='我的英文還可以'),
#                                                                                                     MessageAction(
#                                                                                                         label='低',
#                                                                                                         text='我的英文還需加強'),                                                                                              
#                                                                                                  ]))])
#     elif (event.message.text.find('我的英文頂呱呱')== 0):       
#         line_bot_api.reply_message(event.reply_token, [
#                                                   TemplateSendMessage(
#                                                                       alt_text='是否確認進入真人諮詢服務?',
#                                                                       template=ConfirmTemplate(
#                                                                                                 text='是否確認進入真人諮詢服務?',
#                                                                                                 actions=[
#                                                                                                     MessageAction(
#                                                                                                         label='是',
#                                                                                                         text='呼叫小編!!!'),
#                                                                                                     MessageAction(
#                                                                                                         label='否',
#                                                                                                         text='請點選下方打字機圖片點選查看更多資訊')
#                                                                                                 ]))])